# Lesson 6 - Creating an Healthcare Provider Agent using LangGraph, MCP, and A2A

In this lesson, you will build a third agent: a Healthcare Provider Agent. This agent demonstrates a powerful combination of technologies:
1.  **MCP (Model Context Protocol)**: You will build a server that exposes a tool to find doctors from a JSON file.
2.  **LangGraph**: You will build an agent that uses this MCP tool.
3.  **A2A**: You will wrap the LangGraph agent in an A2A server.
4.  **LiteLLM**: You will use LiteLLM to interact with the Google Gemini API.


## 6.1. Create the MCP Server

The MCP server code is provided in `agents/mcpserver.py`. This server exposes a tool called `list_doctors` which queries a local `doctors.json` file.


In [ ]:
from IPython.display import Code, display

display(Code("agents/mcpserver.py"))

## 6.2. Test with LangGraph and MCP Client

Now you will verify that you can connect to the MCP server and use it within a LangChain/LangGraph agent. You use `MultiServerMCPClient` to connect to the local script via `stdio` transport.


In [ ]:
import asyncio

from IPython.display import Markdown, display
from langchain.agents import create_agent
from langchain_litellm import ChatLiteLLM
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.sessions import StdioConnection

from agents.helpers import setup_env

setup_env()

In [ ]:
mcp_client = MultiServerMCPClient(
    {
        "find_healthcare_providers": StdioConnection(
            transport="stdio",
            command="uv",
            args=["run", "mcpserver.py"],
        )
    }
)

In [ ]:
agent = create_agent(
    ChatLiteLLM(
        model="gemini/gemini-3-flash-preview",
        max_tokens=1000,
    ),
    asyncio.run(mcp_client.get_tools()),
    name="HealthcareProviderAgent",
    system_prompt="Your task is to find and list healthcare providers using the find_healthcare_providers MCP Tool based on the users query.",
)

In [ ]:
prompt = "I'm based in Austin, TX. Are there any Psychiatrists near me?"
response = await agent.ainvoke(
    {
        "messages": [
            {
                "role": "user",
                "content": prompt,
            }
        ]
    }
)

In [ ]:
display(Markdown(response["messages"][-1].content))

## 6.3. Wrap in A2A Server

We have encapsulated this logic in `agents/a2a_provider_agent.py`. This file handles the MCP client connection and agent initialization and wraps the LangGraph agent as an A2A server using the [`langgraph-a2a-server`](https://github.com/5enxia/langgraph-a2a-server/) library.

In [ ]:
from IPython.display import Code, display

display(Code("agents/a2a_provider_agent.py"))

## 6.4. Run the Provider Server

Activate the Provider Agent in Terminal 3.
- Open a terminal as instructed below.
- Type `uv run agents/a2a_provider_agent.py`

<div style="background-color:#e8f0fe; padding:15px; border-left:5px solid #4285f4; border-radius:4px">
    <b>Terminal Access:</b> Please open a new terminal window in your Jupyter environment to run the server.
    <br>You can typically do this by selecting <i>File -> New -> Terminal</i> from the menu.
</div>

## 6.7. Resources

- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)
- [LangChain MCP Adapters](https://docs.langchain.com/oss/python/langchain/mcp)
- [LiteLLM](https://docs.litellm.ai/)


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>